In [ ]:
import tensorflow as tf
import numpy as np
import math

In [ ]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.W = tf.Variable(tf.random.uniform(tf.TensorShape((input_size, output_size)), minval=0, maxval=1e-1))
        self.b = tf.Variable(tf.zeros(tf.TensorShape(output_size,)))
        self.activation = activation
        
    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    def weights(self):
        return [self.W, self.b]

In [ ]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
        
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [ ]:
class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)
        
    def next(self):
        images = self.images[self.index: self.index + self.batch_size]
        labels = self.labels[self.index: self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [ ]:
def train_one_epoch(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        loss = tf.losses.sparse_categorical_crossentropy(labels_batch, predictions) / len(images_batch)
        gradients = tape.gradient(loss, model.weights)
        update_weights(gradients, model.weights)
        return loss

In [ ]:
optimizer = tf.keras.optimizers.SGD(1e-3)

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [ ]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch in range(epochs):
        print('Epoch:', epoch, '/', epochs)
        generator = BatchGenerator(images, labels, batch_size=batch_size)
        losses = []
        for batch in range(generator.num_batches):
            print('Batch:', batch, '/', generator.num_batches, end='\r')
            images_batch, labels_batch = generator.next()
            loss = train_one_epoch(model, images_batch, labels_batch)
            losses.append(tf.reduce_mean(loss))
        print('Loss:', tf.reduce_mean(losses).numpy())

In [ ]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax),
])

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(60000, 28 * 28).astype("float32") / 255
test_images = test_images.reshape(10000, 28 * 28).astype("float32") / 255

fit(model, train_images, train_labels, 10)

In [ ]:
predictions = model(test_images)
predicted_labels = np.argmax(predictions.numpy(), axis=1)
matches = predicted_labels == test_labels
print(np.mean(matches))